In [1]:
import pandas as pd
from pyalex import Works, Authors, Sources, Institutions, Concepts, Publishers, Funders
from thefuzz import fuzz
import json
import re
import urllib.parse

In [2]:
df_metadata_trec = pd.read_parquet("../../../data/metadata_TREC.parquet")

In [3]:
with open("../../../data/SemanticScholar_TREC.json", encoding="utf-8") as f:
    SemanticScholar_FINAL = json.load(f)

In [6]:
# Initialize lists to store data

merge_list = [] # Will contain pairs of Semantic Scholar IDs and corresponding OpenAlex IDs

# Iterate through each entry in the SemanticScholar_FINAL dictionary
for i in SemanticScholar_FINAL:
    try:
        # Attempt to extract the OpenAlex ID for each Semantic Scholar entry
        merge_list.append([i, "W" + SemanticScholar_FINAL[i]["externalIds"]["MAG"]]) # Append the MAG ID by Semantic Scholar and OpenAlex ID to merge_list
    except:
        # If there's a KeyError, it means the expected data is missing
        continue

In [8]:
df_has_mag = pd.DataFrame(merge_list, columns=["ID","MAG"])

In [9]:
df_has_mag_merged = pd.merge(df_has_mag, df_metadata_trec, how="inner", left_on="ID", right_on="ID")

In [19]:
def extract_years(title):
    return re.findall(r'\b\d{4}\b', title)

In [12]:
# Initialize dictionaries and lists to store data

OpenAlex_by_MAG = {}  # Will store matched OpenAlex works with Semantic Scholar IDs as keys
ids_found = []        # Will keep track of IDs that were successfully matched

# Iterate through each row in the DataFrame df_has_mag_merged (Already found documents in Semantic Scholar that provide MAG ID)
for i,j in df_has_mag_merged.iterrows():
    try:
        # Fetch the OpenAlex work using the MAG ID from the current row
        work = Works()[j["MAG"]]
        
        # Extract years from the work title and the DataFrame title
        years_semantic = extract_years(work["title"])
        years_df = extract_years(j["Title"])

        # Check if the titles match closely and if the years are the same
        if fuzz.ratio(j["Title"], work["title"]) > 95 and years_df == years_semantic: 
            OpenAlex_by_MAG[j["ID"]] = work # Store the matched work in the dictionary with Semantic Scholar ID as key
            ids_found.append(j["ID"])       # Add the ID to the list of found IDs
    except:
        # Handle cases where the OpenAlex work could not be retrieved or other errors occur
        print("ID not given or found")
        continue


0 / 1605
1 / 1605
2 / 1605
3 / 1605
4 / 1605
5 / 1605
6 / 1605
7 / 1605
8 / 1605
9 / 1605
10 / 1605
11 / 1605
12 / 1605
13 / 1605
14 / 1605
15 / 1605
16 / 1605
17 / 1605
18 / 1605
19 / 1605
20 / 1605
21 / 1605
22 / 1605
23 / 1605
24 / 1605
25 / 1605
26 / 1605
27 / 1605
28 / 1605
29 / 1605
30 / 1605
31 / 1605
32 / 1605
33 / 1605
34 / 1605
35 / 1605
36 / 1605
37 / 1605
38 / 1605
39 / 1605
40 / 1605
41 / 1605
42 / 1605
43 / 1605
44 / 1605
45 / 1605
46 / 1605
47 / 1605
48 / 1605
49 / 1605
50 / 1605
51 / 1605
52 / 1605
53 / 1605
54 / 1605
55 / 1605
56 / 1605
57 / 1605
58 / 1605
59 / 1605
60 / 1605
61 / 1605
62 / 1605
63 / 1605
64 / 1605
65 / 1605
66 / 1605
67 / 1605
68 / 1605
69 / 1605
70 / 1605
71 / 1605
72 / 1605
73 / 1605
74 / 1605
75 / 1605
76 / 1605
77 / 1605
78 / 1605
79 / 1605
80 / 1605
81 / 1605
82 / 1605
83 / 1605
84 / 1605
85 / 1605
86 / 1605
87 / 1605
88 / 1605
89 / 1605
90 / 1605
91 / 1605
92 / 1605
93 / 1605
94 / 1605
95 / 1605
96 / 1605
97 / 1605
98 / 1605
99 / 1605
100 / 1605

In [17]:
# IDs from OpenAlex that we have already found and processed by MAG ID
ids_found_with_MAG = list(OpenAlex_by_MAG.keys())

# Filter out documents from the main DataFrame that have already been processed (i.e., have MAG IDs)
df_no_mag = df_metadata_trec[~df_metadata_trec["ID"].isin(ids_not_found)]

# Dictionary to store documents that need to be matched by title
documents_without_MAG = {}

# Iterate through the filtered DataFrame
for i, j in df_no_mag.iterrows():
    
    # Clean and encode the title for the search query
    title = j["Title"].replace(",", "")
    title_encoded = urllib.parse.quote(title)
    try:
        # Search for works in OpenAlex by the cleaned and encoded title
        work = Works().search_filter(title=title_encoded).get()    
        
        if len(work) == 0:
            # If no results are found, print a message and continue to the next document
            print("no matches")
            continue
        elif len(work) == 1:
            # If exactly one result is found, check if it matches well with the title and year
            print("One option")
            years_semantic = extract_years(work[0]["title"])
            years_df = extract_years(j["Title"])
            if fuzz.ratio(j["Title"], work[0]["title"]) > 95 and years_df == years_semantic:
                # Add the result to the dictionary if the title and year match
                documents_without_MAG[j["ID"]] = work[0]
        elif len(work) > 1:
            # If multiple results are found, display them for user selection
            for k in work:
                print(k["id"])
            print(j["Title"])
            print(j["Authors"])
            
            # Prompt the user to select the correct result
            mag_id_to_take = int(input())
            if mag_id_to_take != "-":
                documents_without_MAG[j["ID"]] = work[mag_id_to_take]
    except:
        continue

        

1 / 569
https://openalex.org/W22376524
https://openalex.org/W1513290917
The TREC-9 Interactive Track Report.
['William R. Hersh' 'Paul Over']


 0


2 / 569
no matches
3 / 569
no matches
4 / 569
One option
5 / 569
One option
6 / 569
no matches
7 / 569
One option
8 / 569
One option
9 / 569
One option
10 / 569
One option
11 / 569
no matches
12 / 569
no matches
13 / 569
https://openalex.org/W2046020226
https://openalex.org/W2916684075
The TREC 2001 Filtering Track Report.
['Stephen E. Robertson' 'Ian Soboroff']


 0


14 / 569
https://openalex.org/W33021439
https://openalex.org/W2916613452
https://openalex.org/W2902475772
The TREC 2001 Interactive Track Report.
['William R. Hersh' 'Paul Over']


...

559 / 569
no matches
560 / 569
no matches
561 / 569
no matches
562 / 569
no matches
563 / 569
https://openalex.org/W3013372641
https://openalex.org/W4287822877
CAsT 2019: The Conversational Assistance Track Overview
['Jeffrey Dalton' 'Chenyan Xiong' 'Jamie Callan']


 -


564 / 569
no matches
565 / 569
https://openalex.org/W3130740619
https://openalex.org/W3011794880
Overview of the TREC 2019 Deep Learning Track
['Nick Craswell' 'Bhaskar Mitra' 'Emine Yilmaz' 'Daniel Campos'
 'Ellen M. Voorhees']


 -


566 / 569
no matches
567 / 569
One option
568 / 569
One option
569 / 569
no matches


## Manually add still missing OpenAlex documents

In [13]:
# Filter records that are in df_no_mag_not_found but not in documents_without_MAG
df_no_mag_missing_records = df_no_mag_not_found[~df_no_mag_not_found["ID"].isin(list(documents_without_MAG.keys()))]

# Dictionary to store records that are still missing
final_missing_records  = {}

# Iterate over each missing record in the filtered dataframe
for i , j in df_no_mag_not_found_still.iterrows():

    # Clean the title by removing commas
    title = j["Title"].replace(",", "")
    # URL-encode the title for the search query
    title_encoded = urllib.parse.quote(title)
    
    # Search for the record using the encoded title
    work = Works().search_filter(title=title_encoded).get()    
        
    if len(work) == 0:
        # If no results are found, prompt the user to provide an ID
        print(j["Title"])
        print(j["PubYear"])
        print(j["Authors"])
        input_id  = input("Enter OpenAlex ID (or '-' to skip): ")
        if input_id  != "-":
            # Store the record in final_missing_records if an ID is provided
            final_missing_records[j["ID"]] = Works()["W" + input_id ]
    

1 / 222
Question Answering with LCC's CHAUCER at TREC 2006.
2006
['Andrew Hickl' 'John Williams' 'Jeremy Bensley' 'Kirk Roberts' 'Ying Shi'
 'Bryan Rink']


 -


2 / 222
3 / 222
4 / 222
5 / 222
6 / 222
7 / 222
University of Glasgow at TREC 2006: Experiments in Terabyte and Enterprise Tracks with Terrier.
2006
['Christina Lioma' 'Craig Macdonald' 'Vassilis Plachouras' 'Jie Peng'
 'Ben He' 'Iadh Ounis']


 -


8 / 222
Report on the TREC 2006 Experiment: Genomics Track.
2006
['Patrick Ruch' 'Frédéric Ehrler' 'Julien Gobeill' 'Imad Tbahriti'
 'Antonio Jimeno-Yepes']


...

 -


220 / 222
221 / 222
TREC CAR Y3: Complex Answer Retrieval Overview
2019
['Laura Dietz' 'John Foley']


 -


222 / 222
NOVASearch at Precision Medicine 2017
2017
['Gonçalo Araújo' 'André Mourão' 'João Magalhães']


 -


In [24]:
counter = 0

for i , j in df_no_mag_not_found_still.iterrows():

    # Clean the title by removing commas
    title = j["Title"].replace(",", "")
    # URL-encode the title for the search query
    title_encoded = urllib.parse.quote(title)
    
    # Search for the record using the encoded title
    work = Works().search_filter(title=title_encoded).get()

    # Check if exactly one result was found
    if len(work) == 1:
        
        # Extract years from the title of the search result and the current record
        years_semantic = extract_years(work[0]["title"])
        years_df = extract_years(j["Title"])

        # Check if the titles are sufficiently similar and the years match
        if fuzz.ratio(j["Title"], work[0]["title"]) > 95 and years_df == years_semantic:
            # Skip processing if the title and years match
            continue
            
        else:
            # If the work has no locations and no 'Source' key, print the work's ID

            if work[0]["locations_count"] == 0 and "Source" not in work[0]:
                print(work[0]["id"]) 
                print("No locations and no 'Source' key found.")
                continue

            else:
                # Print details of the current record and the work, then prompt for a new ID
                print(j["Title"])
                print(j["PubYear"])
                print(j["Authors"])
                print(work[0]["id"])
                # Prompt the user for a new ID if the current one is not satisfactory
                new_id = input()
                if new_id == "-":
                    # Skip if no new ID is provided
                    continue
                else:
                    # Update the last_dict_new with the new ID
                    final_missing_records[j["ID"]] = Works()["W" + new_id]


1 / 222
2 / 222
NTU at TREC 2006 Genomics Track.
2006
['Kevin Hsin-Yih Lin' 'Wen-Juan Hou' 'Hsin-Hsi Chen']
https://openalex.org/W162668321


 162668321


3 / 222
https://openalex.org/W1502658005
keine locations und keine Source
4 / 222
https://openalex.org/W2157553519
keine locations und keine Source
5 / 222
6 / 222
7 / 222
8 / 222
9 / 222
10 / 222
A Concept-Based Framework for Passage Retrieval at Genomics.
2006
['Wei Zhou' 'Clement T. Yu' 'Vetle I. Torvik' 'Neil R. Smalheiser']
https://openalex.org/W108921306


 108921306


11 / 222
https://openalex.org/W2149043723
keine locations und keine Source
12 / 222
The Ephyra QA System at TREC 2006.
2006
['Nico Schlaefer' 'P. Gieselman' 'Guido Sautter']
https://openalex.org/W2152011797


...

158 / 222
159 / 222
160 / 222
161 / 222
162 / 222
163 / 222
164 / 222
165 / 222
Hybrid Retrieval and Multi-stage Ranking at TREC 2022 Deep Learning Track.
2022
['Guangwei Xu' 'Yanzhao Zhang' 'Longhui Zhang' 'Dingkun Long'
 'Pengjun Xie' 'Ruijie Guo']
https://openalex.org/W4386148436


 -


166 / 222
167 / 222
168 / 222
169 / 222
170 / 222
171 / 222
172 / 222
173 / 222
174 / 222
175 / 222
176 / 222
177 / 222
178 / 222
179 / 222
180 / 222
181 / 222
182 / 222
183 / 222
184 / 222
185 / 222
186 / 222
187 / 222
188 / 222
189 / 222
190 / 222
191 / 222
192 / 222
193 / 222
194 / 222
195 / 222
196 / 222
197 / 222
198 / 222
199 / 222
200 / 222
201 / 222
202 / 222
203 / 222
204 / 222
205 / 222
206 / 222
207 / 222
208 / 222
209 / 222
210 / 222
SIB Text Mining at TREC 2018 Precision Medicine Track
2018
['Emilie Pasche' 'Paul van Rijen' 'Julien Gobeill' 'Anaïs Mottaz'
 'Luc Mottin' 'Patrick Ruch']
https://openalex.org/W2988098012


 2988098012


211 / 222
212 / 222
213 / 222
214 / 222
215 / 222
216 / 222
217 / 222
218 / 222
219 / 222
220 / 222
221 / 222
222 / 222


In [79]:
final_missing_records["trec_663"] = Works()["W2141614686"]
final_missing_records["trec_723"] = Works()["W2899701551"]
final_missing_records["trec_1318"] = Works()["W2917201903"]
final_missing_records["trec_1323"] = Works()["W1483624827"]


In [81]:
full_json = OpenAlex_by_MAG | documents_without_MAG | final_missing_records

In [83]:
with open('../../../data/OpenAlex_TREC.json', 'w', encoding='utf-8') as f:
    json.dump(full_json, f, ensure_ascii=False, indent=4)

In [84]:
df_finally_missing = df_metadata_trec[~df_metadata_trec["ID"].isin(list(full_json.keys()))]

In [86]:
df_finally_missing.to_parquet("../../../data/OpenAlex_TREC_not_found.parquet")